# XtEHR Logical model processing

* ophalen XtEHR Logical Models
* consistente layout met zibs
* klaar voor een mapping - zie mapping Notebook

## Relevante kern content

NL kernset:
- Behandelaanwijzing

ePS required:
- Medicatie
- Diagnose/ Probleemlijst(actief)
- Verrichtingen(<6mnd, ≥ 6mnd)
- Allergieën en intoleranties + - - 
- Alerts
- Hulpmiddel
- Patiënt

ePS recommended + optional:
- Vaccinations
- Implants
- Treatment restrictions
- Vital signs
- History of illness
- Pregnancy
- Social history
- Functional status
- Results
- Plan of Care
- Encounters

# Imports

In [1]:
import pandas as pd
import requests
import json
import os
from io import StringIO

# Download JSON

* Dit hoeft maar een keer te gebeuren, bij nieuwe mappings download de JSON die nodig is. 
* Kan ook door directe download 'raw' JSON.
* Dit zijn de XtEHR Logical Models

In [2]:
xtehrlist = ['AlertFlag', 'AllergyIntolerance', 'Condition', 'Device','DeviceUse', 'Medication', 'Organization', 'Patient', 'Procedure']
xtehrbase = 'https://build.fhir.org/ig/Xt-EHR/xt-ehr-common/StructureDefinition-EHDS'
# res = 'AlertFlag'
if (not os.path.exists('xtehr')):
    os.makedirs('xtehr')
for res in xtehrlist:
    print(res)
    url = xtehrbase + res + '.json'
    response = requests.get(url)
    jsondata = response.json()
    with open(os.path.join('xtehr', res + '.json'), 'w') as jsonfile:
        json.dump(jsondata, jsonfile, indent=4)

# Kies en lees het te verwerken Logical Model

* Toon en verifieer dat de uitvoer er is

In [15]:
base = 'xtehr'
file = 'Condition'
url = os.path.join(base, 'downloads', file + '.json')
with open(url) as jsonfile:
    xlm = json.load(jsonfile)['snapshot']['element']
dfx = pd.DataFrame(xlm)
dfx

,id,path,short,definition,min,max,base,isModifier,mapping,type,binding
0,EHDSCondition,EHDSCondition,Condition model,C.15 - EHDS refined base model for A clinical ...,0,*,"{'path': 'Base', 'min': 0, 'max': '*'}",False,"[{'identity': 'rim', 'map': 'n/a'}]",NaN,NaN
1,EHDSCondition.identifier,EHDSCondition.identifier,C.15.1 - Identifier,Condition identifier,0,*,"{'path': 'EHDSCondition.identifier', 'min': 0,...",NaN,NaN,[{'code': 'Identifier'}],NaN
2,EHDSCondition.description,EHDSCondition.description,C.15.2 - Description,Condition specification in narrative form,0,1,"{'path': 'EHDSCondition.description', 'min': 0...",NaN,NaN,[{'code': 'Narrative'}],NaN
3,EHDSCondition.code,EHDSCondition.code,C.15.3 - Code,"Code identifying the condition, problem or dia...",0,1,"{'path': 'EHDSCondition.code', 'min': 0, 'max'...",NaN,NaN,[{'code': 'CodeableConcept'}],"{'strength': 'preferred', 'description': 'ICD-..."
4,EHDSCondition.onsetDate,EHDSCondition.onsetDate,C.15.4 - Onset date,Onset date of a problem/condition,0,1,"{'path': 'EHDSCondition.onsetDate', 'min': 0, ...",NaN,NaN,[{'code': 'dateTime'}],NaN
5,EHDSCondition.endDate,EHDSCondition.endDate,C.15.5 - End date,The date or estimated date that the condition ...,0,1,"{'path': 'EHDSCondition.endDate', 'min': 0, 'm...",NaN,NaN,[{'code': 'dateTime'}],NaN
6,EHDSCondition.category,EHDSCondition.category,C.15.6 - Category,Category or categories of the problem.,0,*,"{'path': 'EHDSCondition.category', 'min': 0, '...",NaN,NaN,[{'code': 'CodeableConcept'}],NaN
7,EHDSCondition.clinicalStatus,EHDSCondition.clinicalStatus,C.15.7 - Clinical status,"Status of the condition/problem (active, resol...",0,1,"{'path': 'EHDSCondition.clinicalStatus', 'min'...",NaN,NaN,[{'code': 'CodeableConcept'}],"{'strength': 'preferred', 'description': 'HL7 ..."
8,EHDSCondition.resolutionCircumstances,EHDSCondition.resolutionCircumstances,C.15.8 - Resolution circumstances,Describes the reason for which the status of t...,0,*,{'path': 'EHDSCondition.resolutionCircumstance...,NaN,NaN,"[{'code': 'CodeableReference', 'targetProfile'...",NaN
9,EHDSCondition.severity,EHDSCondition.severity,C.15.9 - Severity,A subjective assessment of the severity of the...,0,1,"{'path': 'EHDSCondition.severity', 'min': 0, '...",NaN,NaN,[{'code': 'CodeableConcept'}],"{'strength': 'preferred', 'description': 'HL7 ..."


# Opschonen uitvoer zodat zib en xtehr consistent zijn

* een card. veld met n..m
* type (is een JSON object) omzetten naar tekst
* alleen nodige kolommen houden

In [16]:
dfx.insert(2, 'card.', None)
dfx['card.'] = dfx['min'].astype(str) + '..' + dfx['max'].astype(str)
for i, row in dfx.iterrows():
    if 'type' in row:
        # row['type'][0]['code']
        try:
            dfx.at[i, 'type'] = row['type'][0]['code'].split('/')[-1]
        except TypeError:
            pass
dfx= dfx[['id', 'path', 'short', 'definition', 'type', 'card.', 'binding']]
dfx

,id,path,short,definition,type,card.,binding
0,EHDSCondition,EHDSCondition,Condition model,C.15 - EHDS refined base model for A clinical ...,NaN,0..*,NaN
1,EHDSCondition.identifier,EHDSCondition.identifier,C.15.1 - Identifier,Condition identifier,Identifier,0..*,NaN
2,EHDSCondition.description,EHDSCondition.description,C.15.2 - Description,Condition specification in narrative form,Narrative,0..1,NaN
3,EHDSCondition.code,EHDSCondition.code,C.15.3 - Code,"Code identifying the condition, problem or dia...",CodeableConcept,0..1,"{'strength': 'preferred', 'description': 'ICD-..."
4,EHDSCondition.onsetDate,EHDSCondition.onsetDate,C.15.4 - Onset date,Onset date of a problem/condition,dateTime,0..1,NaN
5,EHDSCondition.endDate,EHDSCondition.endDate,C.15.5 - End date,The date or estimated date that the condition ...,dateTime,0..1,NaN
6,EHDSCondition.category,EHDSCondition.category,C.15.6 - Category,Category or categories of the problem.,CodeableConcept,0..*,NaN
7,EHDSCondition.clinicalStatus,EHDSCondition.clinicalStatus,C.15.7 - Clinical status,"Status of the condition/problem (active, resol...",CodeableConcept,0..1,"{'strength': 'preferred', 'description': 'HL7 ..."
8,EHDSCondition.resolutionCircumstances,EHDSCondition.resolutionCircumstances,C.15.8 - Resolution circumstances,Describes the reason for which the status of t...,CodeableReference,0..*,NaN
9,EHDSCondition.severity,EHDSCondition.severity,C.15.9 - Severity,A subjective assessment of the severity of the...,CodeableConcept,0..1,"{'strength': 'preferred', 'description': 'HL7 ..."


# Opslaan in Excel

In [17]:
url = os.path.join(base, 'mappable', file + '.xlsx')
dfx.to_excel(url, index=False)